# Exercise 2 -- Group 53 -- Question 34

In [2]:
import numpy as np 
import scipy as sp 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn as sk

print("All packages loaded successfully.")


All packages loaded successfully.


In [ ]:
pd.set_option('display.max_rows', None)

df = pd.read_csv("datasets/bgg_dataset.csv", sep=";")

df["Mechanics"].isnull().sum()

df.loc[df['Mechanics'].isnull(), 'Mechanics'] = 'Unknown'
df.loc[df['Domains'].isnull(), 'Domains'] = 'Unknown'

df.dropna(inplace=True)

df['Owned Users'] = df['Owned Users'].astype(int)

df["Rating Average"] = (
    df["Rating Average"].str.replace(",", ".").astype(float)
)

df["Complexity Average"] = (
    df["Complexity Average"].str.replace(",", ".").astype(float)
)

df["Year Published"] = df["Year Published"].astype(int)

df.loc[df["Year Published"] < 0, "Year Published"] = 0

df.drop("ID", axis=1, inplace=True)

display(df.head())

print(df["Users Rated"].min())

df.info()

# df["Mechanics"].isnull().sum()

,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,Gloomhaven,2017,1,4,120,14,42055,8.79,1,3.86,68323,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,Pandemic Legacy: Season 1,2015,2,4,60,13,41643,8.61,2,2.84,65294,"Action Points, Cooperative Game, Hand Manageme...","Strategy Games, Thematic Games"
2,Brass: Birmingham,2018,2,4,120,14,19217,8.66,3,3.91,28785,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
3,Terraforming Mars,2016,1,5,120,12,64864,8.43,4,3.24,87099,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
4,Twilight Imperium: Fourth Edition,2017,3,6,480,14,13468,8.70,5,4.22,16831,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"


30
<class 'pandas.core.frame.DataFrame'>
Index: 20320 entries, 0 to 20342
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                20320 non-null  object 
 1   Year Published      20320 non-null  int64  
 2   Min Players         20320 non-null  int64  
 3   Max Players         20320 non-null  int64  
 4   Play Time           20320 non-null  int64  
 5   Min Age             20320 non-null  int64  
 6   Users Rated         20320 non-null  int64  
 7   Rating Average      20320 non-null  float64
 8   BGG Rank            20320 non-null  int64  
 9   Complexity Average  20320 non-null  float64
 10  Owned Users         20320 non-null  int64  
 11  Mechanics           20320 non-null  object 
 12  Domains             20320 non-null  object 
dtypes: float64(2), int64(8), object(3)
memory usage: 2.2+ MB
